
Copyright (C) 2017 The HDF Group  
All rights reserved

This example code illustrates how to access a GES DISC ACOS Swath HDF5 file 
with Python via OPeNDAP and visualize it with Panoply and IDV.

If you have any questions, suggestions, or comments on this example, please use
the HDF-EOS Forum (http://hdfeos.org/forums).  If you would like to see an
example of any other NASA HDF/HDF-EOS data product that is not listed in the
HDF-EOS Comprehensive Examples page (http://hdfeos.org/zoo), feel free to
contact us at eoshelp@hdfgroup.org or post it at the HDF-EOS Forum
(http://hdfeos.org/forums).

# Access ACOS data from GES DISC via OPeNDAP

 If you try to access ACOS OPeNDAP data with Panoply or IDV, you will get an error message *URI Too Large* like below. 
 
 ![png](acos_L2s_panoply_error.png)
 
 This example will illustrate how to subset and download data from ACOS OPeNDAP data and save it as netCDF4. Then, we will augment the subsetted and downloaded data so that visualization tool like Panoply and IDV can understand the CF conventions and visualize data as shown below.
 
 ![gif](acos_L2s_idv_animation.gif)
 
(Use nbviewer to see the animated GIF: https://nbviewer.jupyter.org/github/OPENDAP/cloudydap/blob/master/python/acos_L2s.ipynb).
 
 The first step is to make sure that your NASA Earthdata Login username and password works with OPeNDAP server.



In [1]:
from netCDF4 import Dataset # for writing data in netCDF-4/HDF5
from pydap.client import open_url, open_dods
from pydap.cas.urs import setup_session
url = 'https://oco2.gesdisc.eosdis.nasa.gov/opendap/ACOS_L2S.7.3/2016/001/acos_L2s_160101_01_Production_v201201_L2Sub7309_r01_PolB_161028201935.h5'

# Replace username and passowrd to match yours.
session = setup_session('eoshelp', '********', check_url=url)
dataset = open_url(url, session=session)
print dataset

<DatasetType with children 'ABandCloudScreen_albedo_o2_cld', 'ABandCloudScreen_dispersion_multiplier_cld', 'ABandCloudScreen_noise_o2_cld', 'ABandCloudScreen_reduced_chi_squared_o2_cld', 'ABandCloudScreen_reduced_chi_squared_o2_threshold_cld', 'ABandCloudScreen_signal_o2_cld', 'ABandCloudScreen_snr_o2_cld', 'ABandCloudScreen_surface_pressure_apriori_cld', 'ABandCloudScreen_surface_pressure_cld', 'ABandCloudScreen_surface_pressure_delta_cld', 'ABandCloudScreen_surface_pressure_offset_cld', 'ABandCloudScreen_temperature_offset_cld', 'IMAPDOASPreprocessing_ch4_column_apriori_idp', 'IMAPDOASPreprocessing_ch4_column_idp', 'IMAPDOASPreprocessing_ch4_column_uncert_idp', 'IMAPDOASPreprocessing_ch4_weak_band_processing_flag_idp', 'IMAPDOASPreprocessing_cloud_flag_idp', 'IMAPDOASPreprocessing_co2_column_apriori_idp', 'IMAPDOASPreprocessing_co2_column_ch4_window_idp', 'IMAPDOASPreprocessing_co2_column_strong_band_idp', 'IMAPDOASPreprocessing_co2_column_strong_band_uncert_idp', 'IMAPDOASPreprocess

We are interested in time, altitude, lat, lon data along with co2. Subset them only. 

In [2]:
latitude = dataset['SoundingGeometry_sounding_latitude'][:]
longitude = dataset['SoundingGeometry_sounding_longitude'][:]
time = dataset['RetrievalHeader_sounding_time_tai93'][:]
altitude = dataset['SoundingGeometry_sounding_altitude'][:]
data =  dataset['RetrievalResults_xco2'][:]
print latitude

<BaseType with data array([ -9.45687485,  -9.45323086,  -9.44804382, -24.55160713,
       -24.5477829 , -24.54277611, -65.99790192, -65.99738312,
       -65.9967804 , -79.46168518, -79.45751953, -79.44987488,
       -77.84957886, -79.60216522, -79.59919739, -79.59563446,
       -81.44069672, -81.43222046, -81.42183685, -83.17164612,
       -83.16862488, -83.1651535 , -80.97970581, -80.97569275,
       -79.38381195, -79.385849  , -79.38697052, -84.23915863,
       -84.23584747, -84.23146057, -84.34416199, -84.34805298,
       -84.35147095, -81.90871429, -81.91001892, -81.91080475,
       -79.43899536, -79.43900299, -79.43818665, -83.45439148,
       -83.45941925, -81.81082153, -81.81719971, -81.82600403,
       -76.42046356, -76.42217255, -76.42575836, -78.35108185,
       -78.35652161, -78.36101532, -79.88899231, -79.89847565,
       -79.90653992, -77.67523956, -77.68431854, -77.69304657,
       -76.4284668 , -76.4347229 , -76.43946075, -74.72257233,
       -74.72516632, -74.72809601, 

Let's rewrite subsetted data as CF trajectory netCDF-4/HDF5 file (test.nc) that Panoply can visualize. netcDF tools are picky about dimension names. Define *obs* and *trajectory* dimensions. The size of *obs* dimension should match the size of lat/lon/time/altitude dimension of ACOS data.

In [3]:
rootgrp = Dataset("test.nc", "w")
trajectory = rootgrp.createDimension("trajectory", 1)
obs = rootgrp.createDimension("obs", latitude.shape[0])

Create 2D variables using the above dimensions.

In [4]:
time_nc = rootgrp.createVariable("time","f4",("trajectory","obs",))
lon_nc = rootgrp.createVariable("lon","f4",("trajectory","obs",))
lat_nc = rootgrp.createVariable("lat","f4",("trajectory","obs",))
z_nc = rootgrp.createVariable("z","f4",("trajectory","obs",))
data_nc = rootgrp.createVariable("RetrievalResults_xco2","f4",("trajectory","obs",))

netCDF-Java based visualization tools like IDV and Panoply heavily rely on attributes that follow the CF conventions. Add them.

In [5]:
lon_nc.units = "degrees_east"
lon_nc.axis = "X"
lat_nc.units = "degrees_north"
lat_nc.axis = "Y"
time_nc.units = "seconds since 1993-1-1 0:0:0"
time_nc.axis = "T"
z_nc.units = "m"
z_nc.axis = "Z"
data_nc.coordinates = "time lat lon z"


All dimension and variables are defined. Let's add data from OPeNDAP server. 

In [6]:
time_nc[:] = time.data
lon_nc[:] = longitude.data
lat_nc[:] = latitude.data
z_nc[:] = altitude.data
data_nc[:] = data.data


For Panoply, you can stop here by closing the netCDF4 file. 

In [7]:
# Don't close if you want to visualize it with IDV properly.
# rootgrp.close() 

Then, you can open *test.nc* file with Panoply. You can visualize trajectory as shown below.

![png](acos_L2s_panoply_trajectory.png)



IDV requires more conventions than Panoply requires to make trajectory visualization work. You need to add extra variable and CF global attributes as follows. For example, IDV doesn't accept the *units* attribute value **Mole Mole^{-1}**.

In [8]:
rootgrp.Conventions = "CF-1.6"
rootgrp.featureType = "trajectory"
rootgrp.cdm_data_type = "Trajectory"
trajectory_nc = rootgrp.createVariable("trajectory","i",("trajectory",))
trajectory_nc[:] = [1]
data_nc.units = "mole mole-1"
rootgrp.close()

You can open the *test.nc* file as **Trajectory Sounding files** in IDV Data Chooser. Then, you can visualize the data easily as shown below. 

![png](acos_L2s_idv_trajectory.png)
